In [1]:
suppressMessages({
    library(tidyverse)
    library(data.table)

})


save_plot <- function(p, fn, w, h){
    for(ext in c(".pdf", ".png")){
        ggsave(filename=paste0(fn,ext), plot=p, width=w, height=h)
    }
}

dir.create('../_m/')
setwd('../_m/')

Warning message in dir.create("../_m/"):
"'../_m' already exists"


In [2]:
#file_list

In [3]:
# create a list of all the filenames in the folder that end with ".csv"
file_list <- Sys.glob('../../custom_universe*/*/_m/*_gene_set_results.gsa.out')
#file_list
file_list <- file_list[file.info(file_list)$size > 0] %>% # remove empty files (no magma enrichment)
                discard(~str_detect(.x, '/custom_universe/blood_traits/_m/Cv*|/custom_universe/psych_general_traits/_m/Cv*')) # remove DE direction of effect from specific analysis for brain region x brain region 
file_list %>% head()

study_name <- gsub('.*.txt_|_gene_set.*','',file_list)
study_name %>% head()


brain_region <- gsub('.*_m/|_deg.genes.txt.*','',file_list) %>%
                gsub('_.*','',.)
brain_region <- lapply(brain_region, toupper) %>% as.character()

brain_region[brain_region == "CVD"] <- "CAUDATExDLPFC"
brain_region[brain_region == "CVH"] <- "CAUDATExHIPPOCAMPUS"

table(brain_region)
head(brain_region)




comparison <- gsub('.*_m/|_deg.genes.txt.*|','',file_list) %>%
                gsub('.*_','',.)

comparison[comparison == "betweensex"] <- "sex_by_sz"
comparison[comparison == "betweensex"] <- "sex_by_sz"



comparison %>% table()



#_betweensex_deg_genes: sex_by_sz

#XvD: region_by_sex

#maleVfemale: maleVfemale

[1] "../../custom_universe/blood_traits/_m/DLPFC_betweensex_deg_genes.txt_BASO_gene_set_results.gsa.out"      
[2] "../../custom_universe/blood_traits/_m/DLPFC_betweensex_deg_genes.txt_EO_gene_set_results.gsa.out"        
[3] "../../custom_universe/blood_traits/_m/DLPFC_betweensex_deg_genes.txt_MONO_gene_set_results.gsa.out"      
[4] "../../custom_universe/blood_traits/_m/DLPFC_betweensex_deg_genes.txt_NEUT_gene_set_results.gsa.out"      
[5] "../../custom_universe/blood_traits/_m/Hippocampus_betweensex_deg_genes.txt_BASO_gene_set_results.gsa.out"
[6] "../../custom_universe/blood_traits/_m/Hippocampus_betweensex_deg_genes.txt_EO_gene_set_results.gsa.out"

[1] "BASO" "EO"   "MONO" "NEUT" "BASO" "EO"

brain_region
            CAUDATE       CAUDATExDLPFC CAUDATExHIPPOCAMPUS               DLPFC 
                 12                  12                  11                  22 
        HIPPOCAMPUS 
                 22 

[1] "DLPFC"       "DLPFC"       "DLPFC"       "DLPFC"       "HIPPOCAMPUS"
[6] "HIPPOCAMPUS"

.
        all maleVfemale   sex_by_sz 
         23          34          22 

In [4]:
df <- data.frame()

for (i in seq_along(file_list)) {
    dftmp <- fread(file_list[i], skip = 4) %>%
        set_names(c('VARIABLE', 'TYPE', 'NGENES', 'BETA', 'BETA_STD', 'SE', 'P')) %>%
        mutate(GWAS_set = study_name[i],
               Status = ifelse(grepl('down', VARIABLE), 'Downregulated', 'Upregulated'),
               Brain_Region = brain_region[i],
               DEComparison = comparison[i]
        ) %>%
        mutate(GWAS_set = ifelse(GWAS_set == 'SCZ', 'SCHIZOPHRENIA', GWAS_set)) %>%
        group_by(Brain_Region, DEComparison) %>%
        mutate(fdr_bh = p.adjust(P, method = "BH"),
               fdr_bonferroni = p.adjust(P, method = "bonferroni")) %>%
        select(VARIABLE,TYPE, NGENES, BETA, BETA_STD, SE, P, GWAS_set, fdr_bh, fdr_bonferroni, Status ,Brain_Region, DEComparison)
    df <- rbind(dftmp, df) %>% arrange(GWAS_set)
}

df <- df %>% 
        mutate(Status = ifelse(DEComparison == 'all', 'Both', Status),
               DEComparison = ifelse(DEComparison == 'all','region_by_sex', DEComparison))


head(df)
fwrite(df, 'magma_enrichment_results.tsv', quote = F, sep = '\t', row.names = F)


VARIABLE,TYPE,NGENES,BETA,BETA_STD,SE,P,GWAS_set,fdr_bh,fdr_bonferroni,Status,Brain_Region,DEComparison
<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>
CvH__all,SET,41,-0.063564,-0.0026278,0.148890,0.66528,ADHD,0.66528,0.66528,Both,CAUDATExHIPPOCAMPUS,region_by_sex
CvD__all,SET,439,0.044098,0.0059400,0.044584,0.16132,ADHD,0.16132,0.16132,Both,CAUDATExDLPFC,region_by_sex
hippocampus__down,SET,30,-0.127910,-0.0044185,0.188940,0.75078,ADHD,0.89514,1.00000,Downregulated,HIPPOCAMPUS,maleVfemale
hippocampus__up,SET,30,-0.222350,-0.0076808,0.177250,0.89514,ADHD,0.89514,1.00000,Upregulated,HIPPOCAMPUS,maleVfemale
dlpfc__down,SET,75,0.063748,0.0034851,0.118520,0.29534,ADHD,0.59068,0.59068,Downregulated,DLPFC,maleVfemale
dlpfc__up,SET,102,-0.040100,-0.0025552,0.099966,0.65584,ADHD,0.65584,1.00000,Upregulated,DLPFC,maleVfemale


In [5]:
subset(df , P < 0.05)

VARIABLE,TYPE,NGENES,BETA,BETA_STD,SE,P,GWAS_set,fdr_bh,fdr_bonferroni,Status,Brain_Region,DEComparison
<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>
Male__up__Hippocampus,SET,66,0.20254,0.010313,0.11082,0.0338110,BASO,0.0676220,0.0676220,Upregulated,HIPPOCAMPUS,sex_by_sz
Male__down__DLPFC,SET,4,1.09770,0.013801,0.47235,0.0100680,BASO,0.0100680,0.0100680,Downregulated,DLPFC,sex_by_sz
Male__up__Hippocampus,SET,65,0.31140,0.015935,0.18339,0.0447550,BMI,0.0895100,0.0895100,Upregulated,HIPPOCAMPUS,sex_by_sz
Male__down__DLPFC,SET,4,1.30200,0.016369,0.56729,0.0108700,EO,0.0108700,0.0108700,Downregulated,DLPFC,sex_by_sz
Male__up__Hippocampus,SET,66,0.30420,0.015489,0.12965,0.0094876,NEUT,0.0189752,0.0189752,Upregulated,HIPPOCAMPUS,sex_by_sz


In [6]:
df %>% arrange(desc(abs(BETA))) %>% head(20)

VARIABLE,TYPE,NGENES,BETA,BETA_STD,SE,P,GWAS_set,fdr_bh,fdr_bonferroni,Status,Brain_Region,DEComparison
<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>
Male__down__DLPFC,SET,4,1.30200,0.0163690,0.56729,0.0108700,EO,0.0108700,0.0108700,Downregulated,DLPFC,sex_by_sz
Male__down__DLPFC,SET,4,1.22730,0.0155310,0.97708,0.1045600,HEIGHT,0.1045600,0.1045600,Downregulated,DLPFC,sex_by_sz
Male__down__DLPFC,SET,4,1.09770,0.0138010,0.47235,0.0100680,BASO,0.0100680,0.0100680,Downregulated,DLPFC,sex_by_sz
Male__down__DLPFC,SET,4,0.79598,0.0100070,0.59805,0.0916110,MONO,0.0916110,0.0916110,Downregulated,DLPFC,sex_by_sz
Male__down__DLPFC,SET,4,-0.66036,-0.0083493,0.47647,0.9171100,ADHD,0.9171100,0.9171100,Downregulated,DLPFC,sex_by_sz
hippocampus__down,SET,30,-0.49956,-0.0172720,0.34626,0.9254400,HEIGHT,0.9254400,1.0000000,Downregulated,HIPPOCAMPUS,maleVfemale
CvH__all,SET,41,0.42939,0.0177670,0.27638,0.0601470,HEIGHT,0.0601470,0.0601470,Both,CAUDATExHIPPOCAMPUS,region_by_sex
hippocampus__up,SET,31,-0.37529,-0.0131060,0.17701,0.9830000,BIPOLAR,0.9830000,1.0000000,Upregulated,HIPPOCAMPUS,maleVfemale
CvH__all,SET,42,-0.35296,-0.0146870,0.14504,0.9925200,BASO,0.9925200,0.9925200,Both,CAUDATExHIPPOCAMPUS,region_by_sex


In [7]:
sessionInfo()

R version 4.1.2 (2021-11-01)
Platform: x86_64-pc-linux-gnu (64-bit)
Running under: Arch Linux

Matrix products: default
BLAS:   /usr/lib/libopenblasp-r0.3.18.so
LAPACK: /usr/lib/liblapack.so.3.10.0

locale:
[1] C

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
 [1] data.table_1.14.8 lubridate_1.9.2   forcats_1.0.0     stringr_1.5.0    
 [5] dplyr_1.1.2       purrr_1.0.2       readr_2.1.4       tidyr_1.3.0      
 [9] tibble_3.2.1      ggplot2_3.4.2     tidyverse_2.0.0  

loaded via a namespace (and not attached):
 [1] pillar_1.9.0     compiler_4.1.2   base64enc_0.1-3  tools_4.1.2     
 [5] digest_0.6.33    uuid_1.1-0       timechange_0.2.0 jsonlite_1.8.7  
 [9] evaluate_0.21    lifecycle_1.0.3  gtable_0.3.3     pkgconfig_2.0.3 
[13] rlang_1.1.1      IRdisplay_1.1    cli_3.6.1        IRkernel_1.3.2  
[17] fastmap_1.1.1    repr_1.1.6       withr_2.5.0      hms_1.1.3       
[21] generics_0.1.3   vctrs_0.6.3      g